In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
fleetpy_study_dirpath = r"D:\projects\fleetpy\github\ptbroker\studies\j26-tpcs"
fleetpy_study_results_dirpath = os.path.join(fleetpy_study_dirpath, "results")
fleetpy_study_scenarios_dirpath = os.path.join(fleetpy_study_dirpath, "scenarios")

mandatory_result_files = [
    "00_config.json",
    "00_simulation.log",
    "1_user-stats.csv",
    "2_vehicle_types.csv",
    "2-0_op-stats.csv",
    "3-0_op-dyn_atts.csv",
    "final_state.csv",
]

simulation_progress_log_filepath = os.path.join(fleetpy_study_scenarios_dirpath, "simulation_progress.log")

# Read all scenarios
all_scenarios_df = pd.read_csv(os.path.join(fleetpy_study_scenarios_dirpath, "amod_scenario_configs_20251006.csv"))
all_scenario_names = all_scenarios_df['scenario_name'].tolist()

# Find unprocessed scenario names
unprocessed_scenarios = []
for scenario_name in tqdm(all_scenario_names):
    scenario_dirpath = os.path.join(fleetpy_study_results_dirpath, scenario_name)
    if not os.path.exists(scenario_dirpath):
        unprocessed_scenarios.append(scenario_name)
    elif not all(os.path.exists(os.path.join(scenario_dirpath, f)) for f in mandatory_result_files):
        unprocessed_scenarios.append(scenario_name)

print(f"Found {len(unprocessed_scenarios)} unprocessed scenarios out of {len(all_scenario_names)} total scenarios.")
# Create a DataFrame for unprocessed scenarios
unprocessed_scenarios_df = all_scenarios_df[all_scenarios_df['scenario_name'].isin(unprocessed_scenarios)]
unprocessed_scenarios_df.reset_index(drop=True, inplace=True)

# Cross-check with simulation progress log
if os.path.exists(simulation_progress_log_filepath):
    with open(simulation_progress_log_filepath, 'r') as f:
        processed_scenarios_in_log = [line.split()[0] for line in f if line.strip()]
    unprocessed_scenarios_in_log_df = all_scenarios_df[~all_scenarios_df['scenario_name'].isin(processed_scenarios_in_log)]
    unprocessed_scenarios_in_log_df.reset_index(drop=True, inplace=True)
    print(f"Found {len(unprocessed_scenarios_in_log_df)} unprocessed scenarios in log out of {len(all_scenario_names)} total scenarios.")

    # Merge with previously found unprocessed scenarios
    all_unprocessed_scenarios_df = pd.concat([unprocessed_scenarios_df, unprocessed_scenarios_in_log_df]).drop_duplicates().reset_index(drop=True)
    print(f"Total unique unprocessed scenarios: {len(all_unprocessed_scenarios_df)}")

    # Save to CSV
    all_unprocessed_scenarios_df.to_csv(os.path.join(fleetpy_study_dirpath, "scenarios", "amod_scenario_configs_20251006_unprocessed.csv"), index=False)

100%|██████████| 126000/126000 [00:12<00:00, 10398.15it/s]

Found 17101 unprocessed scenarios out of 126000 total scenarios.
